In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)
library(data.table)
library(dplyr)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Wang/3.Genome_wide_prediction/SCENT/SCENT_2500.250324/pairs.E2G.res.rds"
path.pairs.E2G.reuse = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Wang/3.Genome_wide_prediction/SCENT/SCENT_25000.240612/pairs.E2G.res.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Wang/K562/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Wang/1.prepare_data/1.seurat_pipeline.240507/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Wang/3.Genome_wide_prediction/SCENT/SCENT_25000.250329.reuse/"
celltype = "K562"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

In [5]:
pairs.E2G.reuse = readRDS(path.pairs.E2G.reuse)

Import ATAC matrix

In [6]:
matrix.atac = read.csv(path.matrix.atac_count,
                       row.names = 1,
                       check.names = F)
matrix.atac = Matrix(as.matrix(matrix.atac), sparse = TRUE)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [7]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [8]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [9]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

Prepare SCENT meta data

In [10]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = celltype)
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
K562_Wang_AAACAGCCAATGCCCG-1,K562_Wang_AAACAGCCAATGCCCG-1,12561,794,K562,9.438352,0.063211528
K562_Wang_AAACAGCCACCCTCAC-1,K562_Wang_AAACAGCCACCCTCAC-1,25127,3064,K562,10.131698,0.121940542
K562_Wang_AAACAGCCACCTACTT-1,K562_Wang_AAACAGCCACCTACTT-1,18172,2314,K562,9.807637,0.127338763
K562_Wang_AAACATGCAACTAACT-1,K562_Wang_AAACATGCAACTAACT-1,14939,982,K562,9.611731,0.065733985
K562_Wang_AAACATGCAAGGTGCA-1,K562_Wang_AAACATGCAAGGTGCA-1,13074,1207,K562,9.478381,0.092320636
K562_Wang_AAACATGCACTAAATC-1,K562_Wang_AAACATGCACTAAATC-1,19262,1239,K562,9.865890,0.064323539
K562_Wang_AAACATGCAGCATGTC-1,K562_Wang_AAACATGCAGCATGTC-1,13940,2080,K562,9.542518,0.149210904
K562_Wang_AAACATGCATTAAGCT-1,K562_Wang_AAACATGCATTAAGCT-1,25225,3039,K562,10.135591,0.120475719
K562_Wang_AAACCAACAAATTGCT-1,K562_Wang_AAACCAACAAATTGCT-1,18599,2216,K562,9.830863,0.119146191


RUN SCENT prediction

In [11]:
RhpcBLASctl::blas_set_num_threads(1)
Sys.setenv(OPENBLAS_NUM_THREADS=1)
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            # bs = boot::boot(df2, assoc, R = 100, formula = formula, 
            #     stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            # p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
                bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        # }
    }
    object@SCENT.result <- res
    return(object)
}

In [12]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [13]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

Filter genes which can be reused

In [14]:
map_gene_names_reuse <- function(gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()
    abc_genes.reuse = abc_genes[abc_genes$abc_name == abc_genes$gene_ref_name,]
	return(abc_genes.reuse)
}

In [15]:
abc_genes.reuse = map_gene_names_reuse(gene_gtf_path, abc_genes_path)

In [16]:
dim(abc_genes.reuse)
abc_genes.reuse

[1] 19669     3

abc_name,Ensembl_ID,gene_ref_name
<chr>,<chr>,<chr>
FAM138A,ENSG00000237613,FAM138A
FAM138F,ENSG00000282591,FAM138F
OR4F5,ENSG00000186092,OR4F5
FAM87B,ENSG00000177757,FAM87B
LINC00115,ENSG00000225880,LINC00115
LINC01128,ENSG00000228794,LINC01128
FAM41C,ENSG00000230368,FAM41C
SAMD11,ENSG00000187634,SAMD11
NOC2L,ENSG00000188976,NOC2L


In [17]:
pairs.E2G.reuse.25000 = pairs.E2G.reuse[pairs.E2G.reuse$boot_basic_p < 0.01 & 
                                        pairs.E2G.reuse$TargetGene %in% abc_genes.reuse$gene_ref_name]

In [18]:
names(pairs.E2G.reuse.25000) = pairs.E2G.reuse.25000$PairName

In [19]:
pairs.E2G.reuse.25000

GRanges object with 16636 ranges and 11 metadata columns:
                                  seqnames              ranges strand |
                                     <Rle>           <IRanges>  <Rle> |
    chr1-10033630-10033792_NMNAT1     chr1   10033630-10033792      * |
  chr1-100351945-100353221_TRMT13     chr1 100351945-100353221      * |
      chr1-10045565-10046681_LZIC     chr1   10045565-10046681      * |
  chr1-10045565-10046681_SLC25A33     chr1   10045565-10046681      * |
    chr1-1004974-1005663_SLC35E2B     chr1     1004974-1005663      * |
                              ...      ...                 ...    ... .
       chrX-75155465-75156980_FTX     chrX   75155465-75156980      * |
      chrX-77910290-77910999_ATRX     chrX   77910290-77910999      * |
     chrX-81121495-81122326_HMGN5     chrX   81121495-81122326      * |
    chrX-85243263-85245260_ZNF711     chrX   85243263-85245260      * |
     chrX-87409179-87410023_KLHL4     chrX   87409179-87410023      * |
      

In [20]:
table(pairs.E2G[pairs.E2G$boot_basic_p < 0.01]$PairName %in% pairs.E2G.reuse.25000$PairName)


FALSE  TRUE 
 3092 15856 

In [21]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr = pairs.E2G[seqnames(pairs.E2G) == chr.tmp]
  pairs.E2G.chr.res.0 = pairs.E2G.chr[!is.na(pairs.E2G.chr$boot_basic_p) & pairs.E2G.chr$boot_basic_p < 0.01]
  pairs.E2G.chr.keep.0 = pairs.E2G.chr[!pairs.E2G.chr$PairName %in% pairs.E2G.chr.res.0$PairName]
  pairs.E2G.chr.res = pairs.E2G.chr.res.0[!pairs.E2G.chr.res.0$PairName %in% pairs.E2G.reuse.25000$PairName]
  pairs.E2G.chr.keep.1 = pairs.E2G.reuse.25000[pairs.E2G.reuse.25000$PairName %in% pairs.E2G.chr.res.0$PairName]

  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(min(n.cores,floor(length(pairs.E2G.chr.res)/2))))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna.rename[rownames(matrix.rna.rename) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(min(n.cores,floor(length(pairs.E2G.chr.res)/2)))
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  pairs.E2G.chr.merge = c(pairs.E2G.chr.res,
                          pairs.E2G.chr.keep.0,
                          pairs.E2G.chr.keep.1)
  saveRDS(pairs.E2G.chr.merge,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.merge)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


...

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [22]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [23]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 628168 ranges and 11 metadata columns:
                                     seqnames              ranges strand |
                                        <Rle>           <IRanges>  <Rle> |
                      chr1.chr1.chr1     chr1 100355009-100355200      * |
                      chr1.chr1.chr1     chr1     1004974-1005663      * |
                      chr1.chr1.chr1     chr1     1020575-1020950      * |
                      chr1.chr1.chr1     chr1     1020575-1020950      * |
                      chr1.chr1.chr1     chr1 103525117-103526544      * |
                                 ...      ...                 ...    ... .
     chrX.chrX-75155465-75156980_FTX     chrX   75155465-75156980      * |
    chrX.chrX-77910290-77910999_ATRX     chrX   77910290-77910999      * |
   chrX.chrX-81121495-81122326_HMGN5     chrX   81121495-81122326      * |
  chrX.chrX-85243263-85245260_ZNF711     chrX   85243263-85245260      * |
   chrX.chrX-87409179-87410023_KLHL4     

In [24]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
df.output[,"FDR"] = p.adjust(df.output$boot_basic_p, method = "BH")
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed,FDR
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100355009,100355200,TRMT13,K562,0.17048567,0.06120206,2.785620,5.342552e-03,0.00680,2.167491,1,2.167491,0.2939809
chr1,1004974,1005663,SDF4,K562,-0.09348173,0.03471736,-2.692651,7.088643e-03,0.01200,1.920819,-1,-1.920819,0.3567785
chr1,1020575,1020950,AGRN,K562,0.16472945,0.06573290,2.506043,1.220909e-02,0.01072,1.969805,1,1.969805,0.3518444
chr1,1020575,1020950,MRPL20,K562,0.11927430,0.03958856,3.012848,2.588088e-03,0.00608,2.216096,1,2.216096,0.2798814
chr1,103525117,103526544,RNPC3,K562,0.08997362,0.03295093,2.730533,6.323198e-03,0.01144,1.941574,1,1.941574,0.3567785
chr1,108691692,108693638,SARS,K562,0.08460655,0.02176077,3.888032,1.010604e-04,0.00192,2.716699,1,2.716699,0.1736120
chr1,108780539,108781988,SORT1,K562,-0.12284575,0.03444453,-3.566480,3.618078e-04,0.01120,1.950782,-1,-1.950782,0.3518444
chr1,108830332,108831023,SARS,K562,0.11238599,0.03087484,3.640051,2.725838e-04,0.00352,2.453457,1,2.453457,0.2262973
chr1,109180710,109182002,PRPF38B,K562,0.08032247,0.02761714,2.908428,3.632504e-03,0.01384,1.858864,1,1.858864,0.3733693


In [25]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4          data.table_1.15.2    Matrix_1.6-5        
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges